In [1]:
from utility_funcs import get_train_labels_test, split_train_data, scale_and_as_array

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
train, labels, test = get_train_labels_test()

In [3]:
features = [f for f in train.columns if 'sensor' in f]

In [4]:
groups = train["sequence"]
train = train.drop(["subject", "step",'sequence'], axis=1)
test = test.drop([ "subject", "step",'sequence'], axis=1)

In [5]:
scaler = StandardScaler()
scaler = scaler.fit(train[features])



In [6]:

X_train, X_valid, y_train, y_valid = split_train_data(train, labels)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

X_train = scale_and_as_array(X_train, features, scaler, scale_data = True)
X_valid = scale_and_as_array(X_valid,  features,scaler, scale_data = True)
X_test = scale_and_as_array(test,  features,scaler, scale_data = True)

(1402260, 13) (23371, 2)
(155820, 13) (2597, 2)


In [7]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_num):
        super().__init__()
        self.X = X
        self.y = y
        self.seq_num = seq_num
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx[0]//self.seq_num]



class TestDataset(torch.utils.data.Dataset):
    def __init__(self, X, seq_num):
        super().__init__()
        self.X = X
        self.seq_num = seq_num
    def __len__(self):
        return len(self.X) // 60
    def __getitem__(self, idx):
        return self.X[idx]



def prepare_data(data, data_labels, seq_num, data_num, mode='train'):
    if data_labels is not None:
        data_labels = data_labels['state'].values
    
    sampler = np.array([list(range(i * seq_num, (i + 1) * seq_num)) for i in range(data_num // seq_num)])
    if mode == 'train':
        dataset = TrainDataset(data, data_labels, seq_num)
    else:
        dataset = TestDataset(data, seq_num)

    dataloader = DataLoader(dataset, batch_size=64, sampler=sampler)
    return dataloader

train_dataloader = prepare_data(X_train, y_train, 60, X_train.shape[0])
valid_dataloader = prepare_data(X_valid, y_valid, 60, X_valid.shape[0])


In [8]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
class LSTM(nn.Module):
    def __init__(self,input_size,num_classes,hidden_size,num_layers):
        super().__init__()
        
        self.lstm1 = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,dropout=0,bidirectional=True)

        self.final = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_size*60*2, num_classes),
        )
            
    def forward(self,x):
        
        out, _ = self.lstm1(x)
        
        out = out.reshape(out.shape[0],-1)
        
        out = self.final(out)
        return out

In [10]:
input_size = len(features)
hidden_size = 128
num_classes = 1
learning_rate = 1e-4
num_epochs = 100

model = LSTM(input_size,num_classes,hidden_size,1)
model.to(device)

LSTM(
  (lstm1): LSTM(13, 128, batch_first=True, bidirectional=True)
  (final): Sequential(
    (0): ReLU()
    (1): Linear(in_features=15360, out_features=1, bias=True)
  )
)

In [11]:

criterion = nn.MSELoss() # seems to preform better than logits
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_warmup_steps = int(0.1 * num_epochs * len(train_dataloader))
num_training_steps = int(num_epochs * len(train_dataloader))
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps
)

In [12]:
torch.cuda.empty_cache()
for epoch in tqdm(range(num_epochs-1)): 
    model.train()
    for trainX, train_y in train_dataloader:
        outputs = model(trainX.to(device,dtype=torch.float32)).squeeze(-1)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        loss = criterion(outputs, train_y.to(device,dtype=torch.float32))
        loss.backward()

        optimizer.step()
        scheduler.step()

    model.eval()
    for validX, valid_y in valid_dataloader:
        with torch.no_grad():
            val_out = model(validX.to(device,dtype=torch.float32)).squeeze(-1)
            vall_loss = criterion(val_out,valid_y.to(device,dtype=torch.float32))

    if epoch % 10 == 0:
        
          print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(epoch, loss.cpu().item(),vall_loss.cpu().item()))

  1%|          | 1/99 [00:06<10:58,  6.72s/it]

Epoch: 0, loss: 0.69522 valid loss:  0.69336 


 11%|█         | 11/99 [01:12<10:16,  7.00s/it]

Epoch: 10, loss: 0.41444 valid loss:  0.40220 


 21%|██        | 21/99 [02:19<08:35,  6.61s/it]

...

 31%|███▏      | 31/99 [03:24<07:27,  6.58s/it]

Epoch: 30, loss: 0.28936 valid loss:  0.28125 


 41%|████▏     | 41/99 [04:31<06:34,  6.80s/it]

Epoch: 40, loss: 0.23694 valid loss:  0.22017 


 42%|████▏     | 42/99 [04:39<06:44,  7.09s/it]

In [ ]:
test_loader = prepare_data(X_test, None, 60, X_test.shape[0], 'test')

KeyError: "None of [Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,\n            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,\n            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,\n            51, 52, 53, 54, 55, 56, 57, 58, 59],\n           dtype='int64')] are in the [columns]"

In [25]:
def predict(
    model,
    loader,
):
    model.eval()
    
    preds = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data.float())
            preds.append(pred.detach().cpu().numpy())

    preds = np.concatenate(preds, 0)
    
    return preds

predict(model, valid_dataloader)

AttributeError: 'list' object has no attribute 'to'